In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL.Image import *
from scipy import ndimage
import matplotlib.image as mpimg

In [ ]:
class HarrisCornerDetector:
    def __init__(self, k = 0.04, lambda_th = 0.01):
        self.k = k
        self.lambda_th = lambda_th
        self.rbg_grey_map = [0.2989, 0.5870, 0.1140]
        
    def rgb2gray(self, rgb_image):
        return np.dot(rgb_image[...,:3], self.rbg_grey_map)
    
    def getMaxEigenValue(self, img):
        rmax = 0
        cols, rows = img.shape
        imarr = np.asarray(img, dtype=np.float64)
        r_eig_vals = np.zeros((cols, rows))

        ix = ndimage.sobel(imarr, 0)
        iy = ndimage.sobel(imarr, 1)

        ix2 = ix * ix
        iy2 = iy * iy
        ixy = ix * iy

        ix2 = ndimage.gaussian_filter(ix2, sigma=2)
        iy2 = ndimage.gaussian_filter(iy2, sigma=2)
        ixy = ndimage.gaussian_filter(ixy, sigma=2)

        for i in range(cols):
            for j in range(rows):
                m = np.array([[ix2[i, j], ixy[i, j]], [ixy[i, j], iy2[i, j]]], dtype=np.float64)
                r_eig_vals[i, j] = np.linalg.det(m) - self.k * (np.power(np.trace(m), 2))
                if r_eig_vals[i, j] > rmax:
                    rmax = r_eig_vals[i, j]
        return rmax, r_eig_vals
    
    def detectHarrisCorners(self, rmax, r_eig_vals):
        r = r_eig_vals
        cols, rows = r_eig_vals.shape
        result = np.zeros((cols, rows))
        for i in range(cols - 1):
            for j in range(rows - 1):
                if r_eig_vals[i, j] > self.lambda_th * rmax and r[i, j] > r[i-1, j-1] and r[i, j] > r[i-1, j+1]\
                                         and r[i, j] > r[i+1, j-1] and r[i, j] > r[i+1, j+1]:
                    result[i, j] = 1
        return result
    
    def plotResults(self, result, img):
        grey_image = rgb2gray(img)
        pc, pr = np.where(result == 1)
        
        f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(15, 20))
        
        ax1.plot(pr, pc, 'r+')
        plt.savefig('harris_test.png')
        ax1.imshow(grey_img, 'gray')
        ax2.imshow(img)
        
        plt.show()

In [ ]:
img = mpimg.imread('../tennis505.ppm')
harris_detector = HarrisCornerDetector(lambda_th = 0.1)

# step-1 conver to grey
grey_img = harris_detector.rgb2gray(img)

rmax, r_eig_vals = harris_detector.getMaxEigenValue(grey_img)
print(f'rmax: {rmax} r_eig_vals shape: {r_eig_vals.shape}')

# step-2
result = harris_detector.detectHarrisCorners(rmax, r_eig_vals)

# step-3
harris_detector.plotResults(result, img)